## Ordinary differential equations (solve_ivp)

Integrating a set of `ordinary differential equations (ODEs)` given _initial conditions_ is another useful example. The function `solve_ivp` is available in `SciPy` for integrating a `first-order vector differential equation`:

$\frac{d\mathbf{y}}{dt}=\mathbf{f}\left(\mathbf{y},t\right)$,

given initial conditions $\mathbf{y}\left(0\right)=y_{0}$, where $\mathbf{y}$ is a length $N$ vector and $\mathbf{f}$ is a mapping from $\mathcal{R}^{N}$ to $\mathcal{R}^{N}$. A `higher-order ordinary differential equation` can always be __reduced__ to a _differential equation_ of this type by introducing __`intermediate derivatives`__ into the $\mathbf{y}$ vector.

For example, suppose it is desired to find the solution to the following second-order differential equation:

$\frac{d^{2}w}{dz^{2}}-zw(z)=0$

with _initial conditions_ $w\left(0\right)=\frac{1}{\sqrt[3]{3^{2}}\Gamma\left(\frac{2}{3}\right)}$ and $\left.\frac{dw}{dz}\right|_{z=0}=-\frac{1}{\sqrt[3]{3}\Gamma\left(\frac{1}{3}\right)}$. It is known that the solution to this _differential equation_ with these `boundary conditions` is the __`Airy function`__

$w=\textrm{Ai}\left(z\right)$,

which gives a means to check the _integrator_ using `special.airy`.

First, convert this `ODE` into _standard form_ by setting $\mathbf{y}=\left[\frac{dw}{dz},w\right]$ and $t=z$. Thus, the _differential equation_ becomes

$\frac{d\mathbf{y}}{dt}=\left[\begin{array}{c} ty_{1}\\ y_{0}\end{array}\right]=\left[\begin{array}{cc} 0 & t\\ 1 & 0\end{array}\right]\left[\begin{array}{c} y_{0}\\ y_{1}\end{array}\right]=\left[\begin{array}{cc} 0 & t\\ 1 & 0\end{array}\right]\mathbf{y}$.

In other words,

$\mathbf{f}\left(\mathbf{y},t\right)=\mathbf{A}\left(t\right)\mathbf{y}$.

As an interesting reminder, if $\mathbf{A}\left(t\right)$ commutes with $\int_{0}^{t}\mathbf{A}\left(\tau\right)\, d\tau$ under __`matrix multiplication`__, then this `linear differential equation` has an exact solution using the __matrix exponential__:

$\mathbf{y}\left(t\right)=\exp\left(\int_{0}^{t}\mathbf{A}\left(\tau\right)d\tau\right)\mathbf{y}\left(0\right)$,

However, in this case, $\mathbf{A}\left(t\right)$ and its integral _do not commute_.

This `differential equation` can be solved using the function `solve_ivp`. It requires the _`derivative`, `fprime`, the `time span [t_start, t_end]` and the initial conditions vector, `y0`_, as input arguments and returns an object whose `y` field is _an array_ with consecutive solution values as columns. The initial conditions are therefore given in the first output column.

In [1]:
from scipy import integrate
from scipy import special
import numpy as np

In [2]:
def func(t, y):
    return [t * y[1], y[0]]

In [3]:
def gradient(t, y):
    return [[0, t], [1, 0]]

In [4]:
y1_0 = 1 / 3 ** (2 / 3) / special.gamma(2 / 3)
y0_0 = -1 / 3 ** (1 / 3) / special.gamma(1 / 3)
y0 = [y0_0, y1_0]
t_span = [0, 4]

In [5]:
sol_1 = integrate.solve_ivp(func, t_span, y0)
print(f"{sol_1.t = }")

sol_1.t = array([0.        , 0.10097672, 1.04643602, 1.91060117, 2.49872472,
       3.08684827, 3.62692846, 4.        ])


As it can be seen `solve_ivp` _determines its time steps_ **automatically** if not specified otherwise. To compare the solution of `solve_ivp` with the `airy` function the _time vector_ created by `solve_ivp` is passed to the `airy` function.

In [6]:
print(f"{sol_1.t[1] = }")
print(f"{special.airy(sol_1.t)[0] = }")

sol_1.t[1] = 0.10097672265100152
special.airy(sol_1.t)[0] = array([0.35502805, 0.328952  , 0.12804768, 0.03995804, 0.01575943,
       0.00562799, 0.00201689, 0.00095156])


The solution of `solve_ivp` with its _standard parameters_ shows a _big deviation_ to the `airy` function. To __minimize__ this deviation, _relative and absolute tolerances_ can be used.

In [7]:
rtol, atol = (1e-8, 1e-8)
sol_2 = integrate.solve_ivp(func, t_span, y0, rtol=rtol, atol=atol)
print(f"{sol_2.y[1][0::6] = }")
print(f"{special.airy(sol_2.t)[0][0::6] = }")

sol_2.y[1][0::6] = array([0.35502805, 0.19145234, 0.06368989, 0.0205917 , 0.00554734,
       0.00106409])
special.airy(sol_2.t)[0][0::6] = array([0.35502805, 0.19145234, 0.06368989, 0.0205917 , 0.00554733,
       0.00106406])


To specify _user defined time points_ for the solution of `solve_ivp`, `solve_ivp` offers __two possibilities__ that can also be used _complementarily_. By passing the `t_eval` option to the function call `solve_ivp` returns the solutions of these time points of `t_eval` in its output.

In [8]:
t = np.linspace(0, 4, 100)
sol_3 = integrate.solve_ivp(func, t_span, y0, t_eval=t)
print(f"{sol_3.y[1] = }")

sol_3.y[1] = array([0.35502805, 0.34457455, 0.33414366, 0.32375857, 0.31344848,
       0.30322957, 0.29311531, 0.28311847, 0.27325111, 0.26352463,
       0.2539497 , 0.24453632, 0.23529379, 0.2262307 , 0.21735498,
       0.20867384, 0.2001938 , 0.1919207 , 0.18385965, 0.17601512,
       0.16839084, 0.16098988, 0.15381458, 0.14686662, 0.14014697,
       0.1336559 , 0.12739294, 0.12134991, 0.11552015, 0.10990153,
       0.10449165, 0.0992878 , 0.09428702, 0.08948607, 0.08488142,
       0.08046929, 0.0762456 , 0.072206  , 0.06834587, 0.0646603 ,
       0.06114413, 0.05779189, 0.05459786, 0.05155603, 0.04866012,
       0.04590358, 0.04327956, 0.04078096, 0.03840242, 0.03614376,
       0.0340012 , 0.03197068, 0.03004811, 0.02822941, 0.02651045,
       0.02488709, 0.02335518, 0.02191055, 0.02054902, 0.01926636,
       0.01805837, 0.01692079, 0.01584938, 0.01484095, 0.01389327,
       0.01300391, 0.0121705 , 0.01139064, 0.01066195, 0.00998207,
       0.00934862, 0.00875925, 0.00821161, 0.0077

If the `jacobian matrix` of function is known, it can be passed to the `solve_ivp` to _achieve better results_. Please be aware however that the _default integration method_ __`RK45`__ _**does not support** jacobian matrices_ and thereby another integration method has to be chosen. One of the integration methods that support a jacobian matrix is the for example the `Radau` method of following example.

In [9]:
sol_4 = integrate.solve_ivp(func, t_span, y0, method="Radau", jac=gradient)
print(f"{sol_4.y[1] = }")

sol_4.y[1] = array([0.35502805, 0.34030634, 0.20473892, 0.11623908, 0.06006125,
       0.0285282 , 0.01255771, 0.00529495, 0.00215195, 0.00112582,
       0.00111641])
